# step 1: Predefined CLass

In [2]:
class Propaganda:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self,sentence,SUBJprop):
        self.sentence = sentence
        self.SUBJprop = SUBJprop
        self.propaganda = self.get_propaganda()

    def get_propaganda(self):
        if self.SUBJprop <=2:
            return Propaganda.NEGATIVE
        elif self.SUBJprop == 3:
            return Propaganda.NEUTRAL
        else:
            return Propaganda.POSITIVE

# step 2: Load Data

In [6]:
import pandas as pd
import numpy as np
reviews = []
data  = pd.read_excel('Data/Dataset.xlsx', engine='openpyxl')
df = pd.DataFrame(data, columns = ['Sentence','SUBJprop'])

# iterate elements of attribute "Sentence" and "SUBJprop" and push to the array "reviews"

for index, row in df.iterrows():
    sentence = row['Sentence']
    prop = row['SUBJprop']
    reviews.append(Review(sentence,prop))

reviews[0].sentence
reviews[0].propaganda



'NEGATIVE'

# step 3: Prep Data (split into train and test set)

In [3]:
from sklearn.model_selection import train_test_split
training, test = train_test_split(reviews, train_size=0.7, random_state = 42 ) 
# len(training)
print(len(test))
print(training[0].propaganda)

developmentSet, testSet = train_test_split(test, train_size=0.5, random_state = 42 ) 
print(len(testSet))
print(len(developmentSet))

991
NEGATIVE
496
495


# step 4: Seperate the attribute, originally our array has text and score. we want them to be a seperate array


In [4]:
train_sentence = [ x.sentence for x  in training]     # return just text
train_propaganda = [x.propaganda for x in training]   # return just sentiment

test_sentence = [x.sentence for x in testSet]
test_propaganda = [x.propaganda for x in testSet]

print(train_sentence[0])
print(train_propaganda[0])
print(test_sentence[2])

bey and serena have been running in the same circles since the 22time grammy winner was a member of destiny’s child
NEGATIVE
now theyre finally dating openly but reps for azalea are still lying about the relationship for some reason


# step 4: Bag Of Words

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_sentence) # return a matrix of either 0 or 1 # train_x is our text

test_x_vectors = vectorizer.transform(test_sentence)

print(train_sentence[0])
print(train_x_vectors[0].toarray())

bey and serena have been running in the same circles since the 22time grammy winner was a member of destiny’s child
[[0 0 0 ... 0 0 0]]


#step 4: TF-IDF

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_sentence) # return a matrix of either 0 or 1 # train_x is our text

test_x_vectors = vectorizer.transform(test_sentence)

print(train_sentence[0])
print(train_x_vectors[0].toarray())

bey and serena have been running in the same circles since the 22time grammy winner was a member of destiny’s child
[[0. 0. 0. ... 0. 0. 0.]]


# step 5: Classification SVM

In [29]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_propaganda)

# test_y[0]
clf_svm.predict(test_x_vectors[1])

array(['NEGATIVE'], dtype='<U8')

# step 5: Decision Tree 

In [30]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_propaganda)

clf_dec.predict(test_x_vectors[1])

array(['NEGATIVE'], dtype='<U8')

# step 5: Logistic Regression

In [31]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_propaganda)

clf_log.predict(test_x_vectors[1])

array(['NEGATIVE'], dtype='<U8')

# step 6: Evaluation

In [33]:
# Mean Accuracy
# For Support Vector Machine
print(clf_svm.score(test_x_vectors,test_propaganda))
# For Decision Tree
print(clf_dec.score(test_x_vectors,test_propaganda))
# For Logistic Regression
print(clf_log.score(test_x_vectors,test_propaganda))

0.8911290322580645
0.844758064516129
0.9173387096774194


In [34]:
# F1 Scores
from sklearn.metrics import f1_score

# For Support Vector Machine
print(f1_score(test_propaganda,clf_svm.predict(test_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEUTRAL,Propaganda.NEGATIVE]))
# trash for negative and neutral labels

# For Support Decision Tree
print(f1_score(test_propaganda,clf_dec.predict(test_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEUTRAL,Propaganda.NEGATIVE]))


# For Logistic Regression
print(f1_score(test_propaganda,clf_log.predict(test_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEUTRAL,Propaganda.NEGATIVE]))


[0.28571429 0.16       0.94462541]
[0.04444444 0.23076923 0.92067039]
[0.         0.06451613 0.95881732]


In [35]:
print(train_propaganda.count(Propaganda.POSITIVE))
print(train_propaganda.count(Propaganda.NEGATIVE))


75
2054


In [36]:
test_set =['this is advertisement','bad book do not buy','horrible waste of time']
new_test = vectorizer.transform(test_set)
clf_svm.predict(new_test)

array(['NEGATIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

# step 7: Tuning our model (with Grid Search)

In [41]:
from sklearn.model_selection import GridSearchCV

# tuned = svm.SVC()
parameters = {'kernel':('linear','rbf'),'C':(1,4,8,16,32,64)}
svc = svm.SVC()
clf = GridSearchCV(svc,parameters,cv=10)
clf.fit(train_x_vectors,train_propaganda)
clf.cv_results_


{'mean_fit_time': array([0.84636238, 1.49004517, 0.91160223, 1.89543235, 0.77847877,
        1.31340368, 0.59762824, 1.54938927, 0.6213877 , 1.52501931,
        0.64342873, 1.3120312 ]),
 'std_fit_time': array([0.28788896, 0.1376422 , 0.16620883, 0.22314054, 0.33003522,
        0.08696788, 0.01741298, 0.30178669, 0.06933135, 0.35702647,
        0.12767459, 0.10210445]),
 'mean_score_time': array([0.07667377, 0.08826575, 0.06387362, 0.11037745, 0.07481987,
        0.08449843, 0.04914193, 0.09488091, 0.05846941, 0.11652124,
        0.05158501, 0.09362845]),
 'std_score_time': array([0.05304824, 0.00270606, 0.01125454, 0.02537366, 0.03722528,
        0.00172844, 0.00186313, 0.01594923, 0.02377313, 0.08558301,
        0.00781735, 0.02116289]),
 'param_C': masked_array(data=[1, 1, 4, 4, 8, 8, 16, 16, 32, 32, 64, 64],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
        fill_value='?',
             dtype=object),

In [46]:
import pandas as pd

df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.846362,0.287889,0.076674,0.053048,1,linear,"{'C': 1, 'kernel': 'linear'}",0.857759,0.857143,0.848485,0.870130,0.839827,0.865801,0.831169,0.861472,0.896104,0.878788,0.860668,0.017817,7
1,1.490045,0.137642,0.088266,0.002706,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.887931,0.891775,0.891775,0.891775,0.887446,0.887446,0.887446,0.887446,0.887446,0.887446,0.888793,0.001957,1
2,0.911602,0.166209,0.063874,0.011255,4,linear,"{'C': 4, 'kernel': 'linear'}",0.857759,0.839827,0.826840,0.861472,0.844156,0.857143,0.835498,0.861472,0.878788,0.870130,0.853308,0.015438,10
3,1.895432,0.223141,0.110377,0.025374,4,rbf,"{'C': 4, 'kernel': 'rbf'}",0.892241,0.891775,0.887446,0.891775,0.887446,0.887446,0.883117,0.887446,0.887446,0.887446,0.888358,0.002663,2
4,0.778479,0.330035,0.074820,0.037225,8,linear,"{'C': 8, 'kernel': 'linear'}",0.849138,0.839827,0.826840,0.865801,0.844156,0.857143,0.835498,0.861472,0.878788,0.870130,0.852879,0.015711,11
5,1.313404,0.086968,0.084498,0.001728,8,rbf,"{'C': 8, 'kernel': 'rbf'}",0.892241,0.891775,0.887446,0.883117,0.887446,0.887446,0.883117,0.887446,0.887446,0.887446,0.887493,0.002814,3
6,0.597628,0.017413,0.049142,0.001863,16,linear,"{'C': 16, 'kernel': 'linear'}",0.857759,0.848485,0.826840,0.861472,0.852814,0.857143,0.839827,0.861472,0.874459,0.874459,0.855473,0.013862,8
7,1.549389,0.301787,0.094881,0.015949,16,rbf,"{'C': 16, 'kernel': 'rbf'}",0.892241,0.887446,0.887446,0.874459,0.887446,0.883117,0.887446,0.887446,0.883117,0.887446,0.885761,0.004481,4
8,0.621388,0.069331,0.058469,0.023773,32,linear,"{'C': 32, 'kernel': 'linear'}",0.853448,0.839827,0.831169,0.861472,0.848485,0.844156,0.844156,0.852814,0.878788,0.878788,0.853310,0.014927,9
9,1.525019,0.357026,0.116521,0.085583,32,rbf,"{'C': 32, 'kernel': 'rbf'}",0.892241,0.887446,0.887446,0.870130,0.887446,0.883117,0.883117,0.883117,0.883117,0.887446,0.884462,0.005557,5


In [45]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.860668
1,1,rbf,0.888793
2,4,linear,0.853308
3,4,rbf,0.888358
4,8,linear,0.852879
5,8,rbf,0.887493
6,16,linear,0.855473
7,16,rbf,0.885761
8,32,linear,0.853310
9,32,rbf,0.884462


In [47]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [39]:
print(clf.score(test_x_vectors,test_propaganda))
# For Decision Tree
print(clf_dec.score(test_x_vectors,test_propaganda))
# For Logistic Regression
print(clf_log.score(test_x_vectors,test_propaganda))

0.9193548387096774
0.844758064516129
0.9173387096774194


In [40]:
# F1 Scores
from sklearn.metrics import f1_score

# For Support Vector Machine
print(f1_score(test_propaganda,clf_svm.predict(test_x_vectors),average = None, 
labels=[Propaganda.POSITIVE,Propaganda.NEUTRAL,Propaganda.NEGATIVE]))


[0.28571429 0.16       0.94462541]
